The code does not exactly work, specifically, some function or gradient is wrong such that the loss is increasing instead of decreasing. However, I did understsand the architecture of the code better and did understand somewhat how to work with tensors and pd.

In [145]:
import math
import time
import torch
import numpy as np
import pandas as pd

In [146]:
# preparing the data
data = pd.read_csv('housing.csv')
x = data.loc[:,["area", "bedrooms", "bathrooms", "stories", "parking"]]
x = x[:].values
y = data.loc[:,"price"]
y = y[:].values
x_train = x[0:400]
x_val = x[400:500]
y_train = y[0:400]
y_val = y[400:500]

In [147]:
print(x_train)
print(x_val)
print(y_train)
print(y_val)

[[7420    4    2    3    2]
 [8960    4    4    4    3]
 [9960    3    2    2    2]
 ...
 [5900    2    1    1    1]
 [3120    3    1    2    1]
 [7350    2    1    1    1]]
[[ 3512     2     1     1     1]
 [ 9500     3     1     2     3]
 [ 5880     2     1     1     0]
 [12944     3     1     1     0]
 [ 4900     3     1     2     0]
 [ 3060     3     1     1     0]
 [ 5320     2     1     1     1]
 [ 2145     3     1     3     0]
 [ 4000     2     1     1     0]
 [ 3185     2     1     1     2]
 [ 3850     3     1     1     0]
 [ 2145     3     1     3     0]
 [ 2610     3     1     2     0]
 [ 1950     3     2     2     0]
 [ 4040     2     1     1     0]
 [ 4785     3     1     2     1]
 [ 3450     3     1     1     2]
 [ 3640     2     1     1     0]
 [ 3500     4     1     2     2]
 [ 4960     4     1     3     0]
 [ 4120     2     1     2     0]
 [ 4750     2     1     1     0]
 [ 3720     2     1     1     0]
 [ 3750     3     1     1     0]
 [ 3100     3     1     2     0]
 

In [148]:
class Module:

    def __init__(self, num_features, lr):
        """Initializes the weights and biases of this model"""
        # initializing w as a column vector, such that matrix operation Xw holds
        self.w = torch.normal(0, 0.01, (num_features, 1), requires_grad=True, dtype=float)
        # b only needs to be one, since for linear regression we want to end up with a single model with a single bias
        self.b = torch.zeros(1, requires_grad=True, dtype=float)
        self.lr = lr

    def forward(self, X):
        """Calculates the output of a minibatch of inputs"""
        return (torch.matmul(X, self.w) + self.b)
    
    def loss(self, y_hat, y):
        """Returns the loss value for a batch of prediction and actual results"""
        l = (y_hat - y) ** 2 / 2
        return l.mean()

    def training_step(self, batch):
        """Accepts a data batch and returns the loss value"""
        # batch is an array of [X, y]
        out = self.forward(batch[0])
        l = self.loss(out, batch[1])
        return l
    
    def update_parameters(self, batch):
        """Updates parameters a little bit towards the right direction"""
        loss = self.training_step(batch)
        with torch.no_grad():
            loss.backward()
            self.w -= self.lr * self.w.grad
            self.b -= self.lr * self.b.grad
            self.w.grad.zero_()
            self.b.grad.zero_()

In [149]:
model = Module(5, 0.01)
print(model.w)
print(model.b)

tensor([[-0.0107],
        [ 0.0016],
        [-0.0185],
        [ 0.0149],
        [-0.0238]], dtype=torch.float64, requires_grad=True)
tensor([0.], dtype=torch.float64, requires_grad=True)


In [150]:
class DataModule:

    def __init__(self, batch_size, x_train, x_val, y_train, y_val):
        self.batch_size = batch_size
        self.x_train = x_train
        self.x_val = x_val
        self.y_train = y_train
        self.y_val = y_val
    
    def train_dataloader(self):
        iters = self.x_train.shape[0] // self.batch_size
        for i in range(0, iters):
            yield [torch.tensor(self.x_train[i:i+self.batch_size, :], requires_grad=True, dtype=float), torch.tensor(self.y_train[i:i+self.batch_size], requires_grad=True, dtype=float).reshape((self.batch_size, 1))]

    def val_dataloader(self):
        iters = self.x_val.shape[0] // self.batch_size
        for i in range(0, iters):
            yield torch.tensor(self.x_val[i:i+self.batch_size, :], requires_grad=True, dtype=float)

In [151]:
data = DataModule(20, x_train, x_val, y_train, y_val)
print(x_train.shape)
print(x_val.shape)
print(y_train.shape)
print(y_val.shape)

(400, 5)
(100, 5)
(400,)
(100,)


In [152]:
# training loop without the fancy Trainer class
real_data = DataModule(10, x_train, x_val, y_train, y_val)
real_model = Module(5, 0.1)
epochs = 1
for i in range(epochs):
    for batch in real_data.train_dataloader():
        print(f"Loss: {real_model.training_step(batch).mean()}")
        real_model.update_parameters(batch)
        print(f"Weights: {real_model.w}")
        print(f"Bias: {real_model.b}")

print(f"Trained Weight: {real_model.w}")
print(f"Trained Bias: {real_model.b}")

Loss: 63681586415571.125
Weights: tensor([[9.7592e+09],
        [4.2623e+06],
        [2.5886e+06],
        [2.9134e+06],
        [2.1885e+06]], dtype=torch.float64, requires_grad=True)
Bias: tensor([1122449.1059], dtype=torch.float64, requires_grad=True)
Loss: 4.552598657178302e+27
Weights: tensor([[-9.3299e+16],
        [-3.4398e+13],
        [-2.0282e+13],
        [-2.1999e+13],
        [-1.6069e+13]], dtype=torch.float64, requires_grad=True)
Bias: tensor([-9.0926e+12], dtype=torch.float64, requires_grad=True)
Loss: 3.9681313060915365e+41
Weights: tensor([[8.5063e+23],
        [3.1780e+20],
        [1.7725e+20],
        [1.8807e+20],
        [1.3973e+20]], dtype=torch.float64, requires_grad=True)
Bias: tensor([8.4165e+19], dtype=torch.float64, requires_grad=True)
Loss: 3.094812267700947e+55
Weights: tensor([[-7.2765e+30],
        [-2.8662e+27],
        [-1.5580e+27],
        [-1.6567e+27],
        [-1.1603e+27]], dtype=torch.float64, requires_grad=True)
Bias: tensor([-7.3835e+26], d